# Week 2: Spike-Stimulus Analysis

This notebook covers methods for analyzing the relationship between neural spiking and stimuli.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats